# Example Multiprocessing Simple: Process y

<i>process_target_y</i> is the target function for process <b>y</b>. It is in a file called 
<br>
<b>example_multiprocessing_simple_process_y.py</b>.


The code for the target function for process <b>x</b> is in a file called 
<br>
<b>example_multiprocessing_simple_process_x.py</b>

The parameter for <i>process_target_y</i> is <b>dict_queues</b> which is a dict of
multiprocessing queues. We use a dict rather than passing queues directly. In this
example the input queues for processes <b>x</b> and <b>y</b> are
<b>dict_queues['x']</b> and <b>dict_queues['y']</b>, respectively.

The steps of this target function are as follows:
<ol>
    <li>
        <b><i>Specify the input queue, <i>Stream.scheduler.input_queue</i>,
        for this process</i></b>. 
        The input queue is obtained from the parameter <i>dict_queues.</i>
        <br>
        <code>
            Stream.scheduler.input_queue = dict_queues['y']
        </code>
    </li>
    <li>
        <b><i>Create the streams in this process.</i></b>.
        <br>
        <code>
            y = Stream(name='y')
        </code>
    </li>
    <li>
        <b><i>Create external streams in this process.</i></b>.
        An external stream is specified by two parameters:
        (a) the name of the stream in the receiving process, and (b) the input queue of
        the receiving process.
        <br>
        <code>
            x = ExternalStream(name='x', queue=dict_queues['x'])
        </code>
    </li>
    <li>
        <b><i>Declare the callback functions used in this process</i></b>. The callback function
        prints the new item of the stream and then appends the incremented item to 
        the external stream <b>x</b>.
        <code>
            print('message received by process y: ', stream_item)
            x.append(stream_item+1)
        </code>
        The callback function first checks whether it has executed <b>count.n</b> times.
        If it has then it informs processes that read its external streams that they can halt; 
        it does so by appending '__halt__' to its external streams.
        The callback function then halts its own process
        by executing <b>Stream.scheduler.halted = True</b>
        <code>
            if not count.n > 0:
                x.append('__halt__')
                Stream.scheduler.halted = True
        </code>
    </li>
    <li>
        <b><i>Create agents in this process</i></b>
        <code>
            single_item(in_stream=y, func=callback_y, count = count)
            # Initiate computation by sending 1 on external stream x
            x.append(1)
        </code>
    </li>
    <li>
        <b><i>Start this process by starting its scheduler</i></b>:
        <code>
            Stream.scheduler.start()
        </code>
    </li>
</ol>

In [1]:
from stream import Stream, ExternalStream
from example_operators import single_item

class Count(object):
    """
    Persistent integer used in callbacks.
    """
    
    def __init__(self, n):  self.n = n

def process_target_y(dict_queues):
    
    #-------------------------------------------
    # 1. SPECIFY INPUT QUEUES FOR THE PROCESSES
    #-------------------------------------------
    Stream.scheduler.input_queue = dict_queues['y']
    
    #-------------------------------------------
    # 2. CREATE STREAMS IN THIS PROCESS
    #-------------------------------------------
    y = Stream(name='y')
    
    #-------------------------------------------
    # 3. CREATE EXTERNAL STREAMS 
    #-------------------------------------------
    x = ExternalStream(name='x', queue=dict_queues['x'])
    
    #-------------------------------------------
    # 4. SPECIFY CALLBACK FUNCTIONS IN THIS PROCESS
    #-------------------------------------------
    
    count = Count(3)
            
    def callback_y(stream_item, count):
        if not count.n > 0:
            x.append('__halt__')
            Stream.scheduler.halted = True
            return
        print('message received by process y: ', stream_item)
        x.append(stream_item+1)
        count.n -= 1
    
    #-------------------------------------------
    # 5. SPECIFY AGENTS IN THIS PROCESS
    #-------------------------------------------
    single_item(in_stream=y, func=callback_y, count = count)
    # Initiate computation by sending 1 on external stream x
    x.append(1)
    
    #-------------------------------------------
    # 6. START SCHEDULER
    #-------------------------------------------
    Stream.scheduler.start()